In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('filtered_data.csv')
df_60, df_40 = train_test_split(df,test_size=0.40, random_state=42)


In [33]:
df_20_ac, df_20 = train_test_split(df_40,test_size=0.50, random_state=42)

In [34]:
df_60.to_csv('federated_data.csv', index=False)
df_20_ac.to_csv('active_learning_data.csv', index=False)
df_20.to_csv('test_data.csv', index=False)

## Save test numpy

In [35]:
import pandas as pd
import numpy as np
import wfdb
import ast
import tensorflow as tf

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    
    data = np.array([signal for signal, meta in data])
    return data
path='ECG_data/'
path_csv = 'test_data.csv'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path_csv)

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)
Y = Y['diagnostic_superclass'].values
# Y = tf.keras.utils.to_categorical(Y,2)


In [36]:
X.shape, Y.shape

((4189, 1000, 12), (4189,))

In [37]:
np.save('X_test.npy', X)
np.save('Y_test.npy', Y)

In [38]:
import numpy as np
from collections import Counter
Counter(Y)

Counter({0: 2364, 1: 1825})

### Active learning part

In [42]:
import pandas as pd
import numpy as np
import tensorflow as tf
import wfdb
from sklearn.model_selection import train_test_split
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    
    data = np.array([signal for signal, meta in data])
    return data


def get_data():

    path='ECG_data/'
    path_csv = 'active_learning_data.csv'
    sampling_rate=100

    # load and convert annotation data
    Y = pd.read_csv(path_csv)
    # Load raw signal data
    X = load_raw_data(Y, sampling_rate, path)
    Y = Y['diagnostic_superclass'].values
    
    return X, Y

In [43]:
X_train, Y_train = get_data()

In [44]:
X_train.shape

(4189, 1000, 12)

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


model = Sequential([
    Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(1000, 12)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
# Train the model
history = model.fit(X_train, Y_train, epochs=20, batch_size=16, verbose=1)


Epoch 1/20
262/262 [==============================] - 2s 6ms/step - loss: 0.5698 - accuracy: 0.8040
Epoch 2/20
262/262 [==============================] - 1s 4ms/step - loss: 0.3744 - accuracy: 0.8341
Epoch 3/20
262/262 [==============================] - 1s 4ms/step - loss: 0.3225 - accuracy: 0.8644
Epoch 4/20
262/262 [==============================] - 1s 4ms/step - loss: 0.2817 - accuracy: 0.8728
Epoch 5/20
262/262 [==============================] - 1s 4ms/step - loss: 0.2545 - accuracy: 0.8978
Epoch 6/20
262/262 [==============================] - 1s 4ms/step - loss: 0.2653 - accuracy: 0.8945
Epoch 7/20
262/262 [==============================] - 1s 4ms/step - loss: 0.2119 - accuracy: 0.9131
Epoch 8/20
262/262 [==============================] - 1s 4ms/step - loss: 0.1991 - accuracy: 0.9231
Epoch 9/20
262/262 [==============================] - 1s 4ms/step - loss: 0.1923 - accuracy: 0.9262
Epoch 10/20
262/262 [==============================] - 1s 4ms/step - loss: 0.1964 - accuracy: 0.9231

In [49]:
import numpy as np
x_test = np.load('X_test.npy')
y_test = np.load('Y_test.npy')
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"loss = {loss}, accuracy: {accuracy}")

loss = 0.5544164776802063, accuracy: 0.8211984038352966


In [50]:
model.save_weights('weights_active.h5')

In [51]:
import pandas as pd

df = pd.read_csv('federated_data.csv')

In [52]:
df

,diagnostic_superclass,filename_lr,filename_hr
0,1,records100/01000/01553_lr,records500/01000/01553_hr
1,0,records100/06000/06835_lr,records500/06000/06835_hr
2,0,records100/14000/14561_lr,records500/14000/14561_hr
3,1,records100/07000/07371_lr,records500/07000/07371_hr
4,0,records100/05000/05478_lr,records500/05000/05478_hr
...,...,...,...
12560,1,records100/11000/11739_lr,records500/11000/11739_hr
12561,1,records100/12000/12466_lr,records500/12000/12466_hr
12562,1,records100/05000/05609_lr,records500/05000/05609_hr
12563,1,records100/00000/00899_lr,records500/00000/00899_hr


In [53]:
import numpy as np
from collections import Counter
Counter(df['diagnostic_superclass'])

Counter({0: 7136, 1: 5429})